# NonlinearModel
* Uses LightGBM to estimate a given (user, item) prediction

In [ ]:
import NBInclude: @nbinclude
@nbinclude("TreeModelBase.ipynb");

In [ ]:
implicit = false;
linear_alphas = ["LinearExplicit", "LinearImplicit"]
all_features = [
    explicit_raw_alphas 
    implicit_raw_alphas 
    nondirectional_raw_alphas    
    linear_alphas
];

In [ ]:
train_model(
    all_features,
    ["LinearExplicit"],
    implicit,
    ["validation"],
    "NonlinearExplicit",
    false,
);

In [ ]:
function save_final_explicit_model()
    linear_splits = reduce(
        cat,
        [read_alpha("LinearExplicit", split, implicit) for split in all_raw_splits],
    )
    nonlinear_splits = reduce(
        cat,
        [read_alpha("NonlinearExplicit", split, implicit) for split in all_raw_splits],
    )
    sparse_preds = sparse(
        linear_splits.user,
        linear_splits.item,
        linear_splits.rating + nonlinear_splits.rating,
    )
    write_alpha(sparse_preds, [], implicit, "Explicit"; log_test_split = true)
    write_params(Dict("alphas" => ["LinearExplicit", "NonlinearExplicit"]), "Explicit")
end;

In [ ]:
save_final_explicit_model()